# Compartimental Model Simulator

**Created by:** Samuel Ropert  
**Creation date:** 04/08/2020  
**Institution:** Computational Biology Lab - Fundación Ciencia y Vida, Chile  

## SEIR
This jupyter notebook shows how to build a single SEIR simulation object with different quarantine scenarios. In this example we show 4 quarantine scenarios, 2 with total quarantines, and 2 with dynamic quarantines.


## Import Libraries

In [1]:
import sys
from pathlib import Path
sys.path.insert(1, '../src/')
sys.path.insert(1, 'src/')
from SEIRmodel import SEIRmodel
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
%matplotlib tk
from joblib import Parallel, delayed
import multiprocessing

## Epidemiological Parameters
* **beta:** Infection rate
* **mu:** Initial exposed obtained from the initial infected mu=E0/I0
* **Scale Factor:** Proportion of real infected compared to reported ones (1: all the infecteds are reported)
* **Sero Prevalence Factor:** Adjust the proportion of the population that enters the virus dynamics
* **Exposed Infection:** rate compared to the infected (0 the don't infect, 1 the infect in the same rate as the infected )

In [2]:
beta = 0.15 # Contagion rate
mu = 0 # E0/I0 initial rate
ScaleFactor = 1 # Scale Factor: Number of real infected over reported 
SeroPrevFactor = 1 # Sero Prevalence Factor: Adjust the proportion of the population that enters the virus dynamics
expinfection = 1 # Exposed contagion rate compared to the infected (0 the don't infect, 1 the infect in the same rate as the infected )

## Simulation Parameters

In [3]:
# Simulation time
tsim = 1000
# Population
population = 1000000
# Initial Active Infected 
I_act0 = 100


### Quarantines

Quarantine Vector:  
 [Tsim, max_mob,rem_mob,quarantine period, quarantine initial time, quarantine final time, quarantine type]

In [4]:
max_mob = 0.8 # Maximum mobility
# Total quarantine
s1 = [tsim,max_mob,0.65,0,0,tsim,0]
s2 = [tsim,max_mob,0.5,0,0,tsim,0]
# Dynamic quarantine
s3 = [tsim,max_mob,0.3,14,0,tsim,1]
s4 = [tsim,max_mob,0.5,14,0,tsim,1]


In [5]:
# Define one quarantine array for each different quarantine remanent mobility
quarantines = [s1,s2,s3,s4]

In [6]:
# Kinetic Saturation: 0 for mass action mixing
k=0

## Create simulation Object

In [7]:
simulation = SEIRmodel(beta = beta,mu = mu,ScaleFactor=ScaleFactor,SeroPrevFactor=SeroPrevFactor,expinfection=expinfection,tsim = tsim,tstate='', k = k)
simulation.inputarray = np.array(quarantines)
simulation.addquarantine()
simulation.initialvalues(I_act0,population,R=0)

Set initial values


## Simulate
The different scenarios are simulated in parallel threads 

In [8]:
simulation.simulate()

SEIR Model
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   1 tasks      | elapsed:    0.8s
[Parallel(n_jobs=8)]: Done   2 out of   4 | elapsed:    0.8s remaining:    0.8s
[Parallel(n_jobs=8)]: Done   4 out of   4 | elapsed:   12.4s remaining:    0.0s
[Parallel(n_jobs=8)]: Done   4 out of   4 | elapsed:   12.4s finished


../src/SEIR_vars.py:143: RuntimeWarning: invalid value encountered in double_scalars
  self.SHFR = [self.totD[i]/(self.I_se_ac[i][-1]+self.I_cr_ac[i][-1]) for i in range(self.numescenarios)]
../src/SEIR_vars.py:144: RuntimeWarning: invalid value encountered in true_divide
  self.SHFR_d = [self.B[i]/(self.I_se_ac[i][-1]+self.I_cr_ac[i][-1]) for i in range(self.numescenarios)]


# Simulation Analysis

## Plots
This libraries have predefined plot functions which plot the main epidemiological variables.
Each function has the following optional arguments:
* days [int] Number of days to display
* showparams [bool] Display simulation parameters 
* ylim [int] Limit the vertical axis
* norm [int/float] Normalize the results


In [9]:
# Epidemiological curves
simulation.plotseir(days = 300)

In [12]:
# Active Infected curves
simulation.plotActiveInfected(showparams=True,days = 650)

In [11]:
# Accumulated infected
simulation.plotAccumulatedInfected()

In [ ]:
# Daily infected
simulation.plotDailyInfected(days=400)

In [ ]:
# Exposed
simulation.plotExposed()

In [ ]:
# Exposed
simulation.plotQuarantines(days=100)

### Variables:
The simulation object contains several methods and variables with its results

In [20]:
# Susceptibles
simulation.S
# Exposed
simulation.E
# Infected
simulation.I
# Recovered
simulation.R


[array([0.00000000e+00, 3.16227666e-05, 6.32455232e-05, 1.22415882e-01,
        2.44619174e-01, 3.66673661e-01, 8.14520573e-01, 1.26038683e+00,
        1.70429843e+00, 2.14628144e+00, 3.65597341e+00, 5.14406805e+00,
        6.61156064e+00, 8.05941180e+00, 9.48854787e+00, 1.27394396e+01,
        1.59047415e+01, 1.89941542e+01, 2.20166075e+01, 2.49803243e+01,
        2.93192579e+01, 3.35665279e+01, 3.77424903e+01, 4.18651624e+01,
        4.53021701e+01, 4.87212725e+01, 5.21302691e+01, 6.38551289e+01,
        6.68114154e+01, 6.97831101e+01, 7.27731790e+01, 7.57842764e+01,
        7.81361942e+01, 8.05033295e+01, 8.28867186e+01, 9.85221886e+01,
        1.15078264e+02, 1.32727863e+02, 1.51617615e+02, 1.69669788e+02,
        1.88916945e+02, 2.09454379e+02, 2.27911857e+02, 2.47422875e+02,
        2.68050114e+02, 2.79408169e+02, 2.91105408e+02, 3.03152090e+02,
        3.45588088e+02, 3.92403656e+02, 4.44025155e+02, 5.28046314e+02,
        6.24676533e+02, 7.35921862e+02, 8.63987626e+02, 1.112331

In [ ]:
# Peak Values per each scenario
simulation.peak

In [ ]:
# Peak time
simulation.peak_t

In [21]:
simulation.showscenarios()

[[1.0e+03 8.0e-01 6.5e-01 0.0e+00 0.0e+00 1.0e+03 0.0e+00]
 [1.0e+03 8.0e-01 5.0e-01 0.0e+00 0.0e+00 1.0e+03 0.0e+00]
 [1.0e+03 8.0e-01 3.0e-01 1.4e+01 0.0e+00 1.0e+03 1.0e+00]
 [1.0e+03 8.0e-01 5.0e-01 1.4e+01 0.0e+00 1.0e+03 1.0e+00]]


()